In [1]:
#ds cleaning before imputation
import os
import pandas as pd
import pyarrow.parquet as pq
import re
import numpy as np
import matplotlib.pyplot as plt

In [8]:

# Define the data folder
#Изменение исходных импутированных датасетов с добавлением метаданных - Condition and Age для последующей регрессии и для рандомного сэмплинга
data_folder = "/tank/projects/vpalagina_hobotnica/hobotnica/data_with_HC_imputed"
output_folder = "/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_for_reg"

# List all parquet files in the data folder
files = [f for f in os.listdir(data_folder)]

# Iterate through the files
for file in files:
    
    # Construct the full file path
    file_path = os.path.join(data_folder, file)
    
    # Read the parquet file
    data = pq.read_table(file_path).to_pandas()
    
    # Check for the presence of the index column and its length
    if data.columns[0] != "index":
        data = data.rename(columns={data.columns[0]: "index"})
        print(f"Warning: First column renamed to 'index' in file: {file}")
    
    # Add condition and Age info to data
    data.set_index('index', inplace=True)
    data = data.transpose()
    data.reset_index(drop=False, inplace=True)

    # Extract datasets information from metadata
    meta = pd.read_csv("computage_bench_meta.tsv", sep="\t", usecols=["SampleID", "Condition", "Age"])
    meta = meta[meta['SampleID'].isin(data['index'])]
    meta = meta.drop_duplicates(subset='SampleID').set_index('SampleID')
    
    # Prepare to merge datasets
    data = data.set_index('index')
    new_ds = data.join(meta, on='index')
    
    # Set a new name
    middle_part = re.search(r"GSE[0-9]+", file).group(0)
    new_filename = f"{middle_part}.pickle"
    dest_file_path = os.path.join(output_folder, new_filename)

    # Save the filtered data
    new_ds.to_pickle(dest_file_path)

In [23]:
# import os
# import pandas as pd

# # Define the folder containing the pickle files
# input_folder = '/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_for_regression_pkl'
# output_folder = '/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_parquet_format'

# for file_name in os.listdir(input_folder):
#     input_file_path = os.path.join(input_folder, file_name)
    
#     # Load the pickle file
#     df = pd.read_pickle(input_file_path)
    
#     output_file_name = file_name.replace('.pickle', '.parquet')
#     output_file_path = os.path.join(output_folder, output_file_name)
    
#     # Save the DataFrame as a Parquet file
#     df.to_parquet(output_file_path)



In [4]:
###Regression for age substracting old method - y - b*age for all datasets

from scipy import stats
from multiprocessing import Pool

# Processing function for linear regression and age
def substract_age(file_path):
    data = pd.read_pickle(file_path)

    x = data['Age']
    def remove_trend(y, x):
        slope, _, _, _, _ = stats.linregress(x, y)
        return y - slope * x

    # Apply the remove_trend function to each column (excluding the last two)
    data.iloc[:, :-2] = data.iloc[:, :-2].apply(remove_trend, x=x)
    return data

    # try:
    #     # Load the data
        # for site in data.columns[:-2]:
        #     y = data[site]
        #     slope, _, _, _, _ = stats.linregress(x, y)
        #     # Subtract the linear trend from the series
        #     data[site] = y - slope * x

        # return data

    # except ValueError as e:
    #     print(f'Error processing {file_path}: {e}')
    #     return None, file_path


def process_and_save(file_path):
    processed_df = substract_age(file_path)
    if processed_df is not None:
        output_path = os.path.join(substracted_age, os.path.basename(file_path))
        processed_df.to_pickle(output_path)
        print(f'Processed and saved: {output_path}')

if __name__ == '__main__':
    initial_datasets = '/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_for_regression_pkl'
    substracted_age = '/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_substracted_age'

    # Get a list of all files in the dataset directory
    files = [os.path.join(initial_datasets, filename) for filename in os.listdir(initial_datasets)]

    # Create a multiprocessing Pool
    with Pool(processes=os.cpu_count()) as pool:
        # Map the files to the processing function
        pool.map(process_and_save, files)

Processed and saved: /tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_substracted_age/GSE56606.pickle
Processed and saved: /tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_substracted_age/GSE49909.pickle


KeyboardInterrupt: 

In [4]:
data1 = pd.read_pickle("/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_for_regression_pkl/GSE32148.pickle")
data2 = pd.read_pickle("/tank/projects/vpalagina_hobotnica/hobotnica/data_imputed_with_meta_substracted_age/GSE32148.pickle")

In [6]:
data1.head()

,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367,rs9839873,Age,Condition
index,,,,,,,,,,,,,,,,,,,,,
GSM796660,0.53599,0.91759,0.85866,0.19967,0.74830,0.61722,0.81706,0.37256,0.35194,0.01726,...,0.40544,0.50740,0.91934,0.52665,0.95103,0.06983,0.50354,0.59608,49.0,IBD
GSM796661,0.51288,0.90251,0.82595,0.19483,0.72959,0.61576,0.82962,0.37086,0.35784,0.02106,...,0.42741,0.49110,0.92444,0.51141,0.94584,0.07401,0.51122,0.59108,49.0,HC
GSM796662,0.50556,0.91070,0.84799,0.19867,0.74411,0.59409,0.80767,0.26378,0.35900,0.01956,...,0.42308,0.51594,0.92105,0.91105,0.52014,0.51397,0.50940,0.90103,43.0,HC
GSM796663,0.53280,0.92180,0.88189,0.17476,0.73424,0.61752,0.82870,0.29243,0.32664,0.01795,...,0.43447,0.51868,0.92616,0.91709,0.53795,0.51322,0.49018,0.89914,43.0,IBD
GSM796664,0.52627,0.91726,0.86505,0.21113,0.74834,0.65703,0.81330,0.27337,0.36234,0.02402,...,0.43955,0.50970,0.48998,0.92633,0.51451,0.92933,0.48832,0.90953,44.0,IBD


In [7]:
data2.head()

,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367,rs9839873,Age,Condition
index,,,,,,,,,,,,,,,,,,,,,
GSM796660,0.637593,0.921337,0.869657,0.194664,0.801906,0.644312,0.819765,0.384437,0.363857,0.015403,...,0.180777,0.375180,1.042524,0.322559,0.824408,0.035778,0.413833,0.502716,49.0,IBD
GSM796661,0.614483,0.906257,0.836947,0.189823,0.783196,0.642852,0.832325,0.382737,0.369757,0.019203,...,0.202747,0.358880,1.047624,0.307319,0.819218,0.039958,0.421513,0.497716,49.0,HC
GSM796662,0.594722,0.913988,0.857640,0.194277,0.791152,0.617865,0.810043,0.274203,0.369458,0.017930,...,0.225927,0.399910,1.029150,0.731950,0.409023,0.484088,0.430678,0.819098,43.0,HC
GSM796663,0.621962,0.925088,0.891540,0.170367,0.781282,0.641295,0.831073,0.302853,0.337098,0.016320,...,0.237317,0.402650,1.034260,0.737989,0.426833,0.483338,0.411458,0.817208,43.0,IBD
GSM796664,0.617505,0.920625,0.874925,0.206634,0.796476,0.681357,0.815729,0.284035,0.373041,0.022352,...,0.237812,0.390972,0.600594,0.743064,0.400809,0.898753,0.407767,0.825693,44.0,IBD
